In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix
import seaborn as sns
import os
import random
import csv
import cv2


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [23]:


def collect_images_recursively(base_folder):
    """
    Recursively collect image paths from all subdirectories.
    """
    image_paths = []
    for root, _, files in os.walk(base_folder):
        for file in sorted(files):
            if file.endswith(('.png', '.jpg', '.jpeg')):  # Add more formats if needed
                image_paths.append(os.path.join(root, file))
    return image_paths

def prepare_dataset(real_faces_folder, real_flow_folder, fake_faces_folder, fake_flow_folder, output_csv):
    """
    Prepare a CSV dataset mapping spatial frames (cropped faces) and optical flow images with labels
    for real and fake data.
    """
    dataset = []

    def process_real_data(cropped_faces_folder, optical_flow_folder, label):
        """
        Process real data, filtering only subdirectories containing "uniform" and "camera_front".
        """
        print("[INFO] Processing real data...")
        for subdir_name in sorted(os.listdir(cropped_faces_folder)):
            # if "uniform" not in subdir_name or "camera_front" not in subdir_name:
            #     continue

            subdir_path = os.path.join(cropped_faces_folder, subdir_name)
            flow_path = os.path.join(optical_flow_folder, subdir_name)

            if not os.path.isdir(subdir_path) or not os.path.isdir(flow_path):
                continue

            spatial_frames = collect_images_recursively(subdir_path)
            optical_flows = collect_images_recursively(flow_path)

            for frame, flow in zip(spatial_frames, optical_flows):
                dataset.append([frame, flow, label])

    def process_fake_data(cropped_faces_folder, optical_flow_folder, label):
        """
        Process fake data, filtering only subdirectories where the first three characters
        of the name after "end_to_end" are numeric and less than 101.
        """
        print("[INFO] Processing fake data...")
        for root, dirs, files in os.walk(cropped_faces_folder):
            if "end_to_end" not in root:
                continue

            # Extract the part after "end_to_end"
            parts = root.split("end_to_end")[-1].strip(os.sep).split(os.sep)
            print(f"[DEBUG] Current root: {root}")
            print(f"[DEBUG] Parts after 'end_to_end': {parts}")
            
            if len(parts) < 1:
                print("[DEBUG] Skipping: No valid parts after 'end_to_end'.")
                continue
            
            # Get the first subdirectory name and split to extract the numeric prefix
            first_part = parts[0]
            numeric_prefix = first_part.split("_")[0]
            
            if not numeric_prefix.isdigit():
                print(f"[DEBUG] Skipping: Prefix '{numeric_prefix}' is not numeric.")
                continue
            
            if int(numeric_prefix) >= 200:
                print(f"[DEBUG] Skipping: Prefix '{numeric_prefix}' >= 101.")
                continue

            # Construct the corresponding optical flow folder
            relative_path = os.path.relpath(root, cropped_faces_folder)
            flow_path = os.path.join(optical_flow_folder, relative_path)
            
            if not os.path.isdir(flow_path):
                print(f"[DEBUG] Skipping: Optical flow folder does not exist for {root}.")
                continue

            # Recursively collect images
            spatial_frames = collect_images_recursively(root)
            optical_flows = collect_images_recursively(flow_path)

            for frame, flow in zip(spatial_frames, optical_flows):
                dataset.append([frame, flow, label])

    # Process real data
    process_real_data(real_faces_folder, real_flow_folder, label=0)

    # Process fake data
    process_fake_data(fake_faces_folder, fake_flow_folder, label=1)

    # Shuffle and save to CSV
    print(f"[INFO] Total dataset size before shuffling: {len(dataset)}")
    random.shuffle(dataset)
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["spatial_frame", "optical_flow", "label"])  # Header
        writer.writerows(dataset)
    print(f"[INFO] CSV saved to: {output_csv}")

# Define paths
real_faces = "E:/Research/real/Cropped_Faces"
real_flow = "E:/Research/real/Optical_Flow"
fake_faces = "E:/Research/fake/Cropped_Faces"
fake_flow = "E:/Research/fake/Optical_Flow"
output_csv = "angry_dataset_real_fake_filtered.csv"

# Prepare the dataset
prepare_dataset(real_faces, real_flow, fake_faces, fake_flow, output_csv)
print("[INFO] Dataset preparation complete.")


[INFO] Processing real data...
[INFO] Processing fake data...
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end
[DEBUG] Parts after 'end_to_end': ['']
[DEBUG] Skipping: Prefix '' is not numeric.
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end\711_M007
[DEBUG] Parts after 'end_to_end': ['711_M007']
[DEBUG] Skipping: Prefix '711' >= 101.
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end\023_M113
[DEBUG] Parts after 'end_to_end': ['023_M113']
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end\759_W025
[DEBUG] Parts after 'end_to_end': ['759_W025']
[DEBUG] Skipping: Prefix '759' >= 101.
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end\952_W101
[DEBUG] Parts after 'end_to_end': ['952_W101']
[DEBUG] Skipping: Prefix '952' >= 101.
[DEBUG] Current root: E:/Research/fake/Cropped_Faces\end_to_end\351_M023
[DEBUG] Parts after 'end_to_end': ['351_M023']
[DEBUG] Skipping: Prefix '351' >= 101.
[DEBUG] Current root: E:/Research/f

In [24]:
# Define CNN for spatial and temporal feature extraction
def create_cnn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation="relu", padding="same")(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation="relu", padding="same")(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    return inputs, x

# Spatial feature extractor
spatial_input, spatial_features = create_cnn((256, 256, 3))  # RGB input

# Temporal feature extractor
temporal_input, temporal_features = create_cnn((256, 256, 3))  # Optical flow input

# Combine features
combined = concatenate([spatial_features, temporal_features])
x = Dense(64, activation="relu")(combined)
x = Dropout(0.5)(x)
x = Dense(32, activation="relu")(x)
x = Dense(1, activation="sigmoid")(x)

# Model
model = Model(inputs=[spatial_input, temporal_input], outputs=x)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"], run_eagerly=False)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_22 (Conv2D)             (None, 256, 256, 8)  224         ['input_3[0][0]']                
                                                                                            

In [25]:
# DataGenerator class
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size, **kwargs):
        super().__init__(**kwargs)  # Ensure compatibility with Keras Sequence API
        self.data = data
        self.batch_size = batch_size

    def __len__(self):
        return len(self.data) // self.batch_size

    def __getitem__(self, idx):
    # Extract a batch from the dataset
        batch = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        spatial_frames = []
        optical_flows = []
        labels = []
        for row in batch:
            spatial_frame_path = row[0]
            optical_flow_path = row[1]
            label = row[2]
    
            # Load spatial frame and optical flow
            spatial_frame = cv2.imread(spatial_frame_path)
            optical_flow = cv2.imread(optical_flow_path)
    
            if spatial_frame is None or optical_flow is None:
                print(f"[WARNING] Missing file(s): {spatial_frame_path} or {optical_flow_path}")
                continue  # Skip missing files
            
            # Resize images to (128, 128, 3)
            spatial_frame = cv2.resize(spatial_frame, (128, 128))
            optical_flow = cv2.resize(optical_flow, (128, 128))
    
            # Normalize images
            spatial_frames.append(spatial_frame / 255.0)
            optical_flows.append(optical_flow / 255.0)
            labels.append(label)
    
        # Convert lists to NumPy arrays
        spatial_frames = np.array(spatial_frames, dtype="float32")
        optical_flows = np.array(optical_flows, dtype="float32")
        labels = np.array(labels, dtype="float32").reshape(-1, 1)  # Ensure shape is (batch_size, 1)
    
        # Convert to TensorFlow tensors explicitly
        spatial_frames = tf.convert_to_tensor(spatial_frames)
        optical_flows = tf.convert_to_tensor(optical_flows)
        labels = tf.convert_to_tensor(labels)
    
        # Return as tuple (for multi-input models)
        return (spatial_frames, optical_flows), labels



In [26]:
# Load the dataset from CSV
dataset = pd.read_csv("angry_dataset_real_fake_filtered.csv").values

# Split into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

In [27]:
# Example batch test
train_gen = DataGenerator(train_data, batch_size=2)
inputs, labels_batch = train_gen[0]
spatial_batch, temporal_batch = inputs
print(f"Spatial batch shape: {spatial_batch.shape}")
print(f"Temporal batch shape: {temporal_batch.shape}")
print(f"Labels batch shape: {labels_batch.shape}")

Spatial batch shape: (2, 128, 128, 3)
Temporal batch shape: (2, 128, 128, 3)
Labels batch shape: (2, 1)


In [28]:
# Load the dataset from the CSV file
dataset = pd.read_csv("angry_dataset_real_fake_filtered.csv").values

# Split into training and validation sets
train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Initialize data generators
train_gen = DataGenerator(train_data, batch_size=32)
val_gen = DataGenerator(val_data, batch_size=32)

# Define a sample model (for example purposes)
input_spatial = tf.keras.layers.Input(shape=(128, 128, 3), name="spatial_input")
input_temporal = tf.keras.layers.Input(shape=(128, 128, 3), name="temporal_input")

# Spatial feature extractor
x1 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_spatial)
x1 = tf.keras.layers.MaxPooling2D((2, 2))(x1)  # Shape: (64, 64, 32)
x1 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x1)
x1 = tf.keras.layers.MaxPooling2D((2, 2))(x1)  # Shape: (32, 32, 64)
x1 = tf.keras.layers.Flatten()(x1)  # Flatten works now because of compatible shape

# Temporal feature extractor
x2 = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_temporal)
x2 = tf.keras.layers.MaxPooling2D((2, 2))(x2)  # Shape: (64, 64, 32)
x2 = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x2)
x2 = tf.keras.layers.MaxPooling2D((2, 2))(x2)  # Shape: (32, 32, 64)
x2 = tf.keras.layers.Flatten()(x2)  # Flatten works now because of compatible shape

# Combine features
combined = tf.keras.layers.Concatenate()([x1, x2])
output = tf.keras.layers.Dense(1, activation="sigmoid")(combined)

# Define the model
model = tf.keras.Model(inputs=[input_spatial, input_temporal], outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"], run_eagerly=False)

# Early Stopping Callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    patience=3,          # Stop training after 3 epochs with no improvement
    restore_best_weights=True  # Restore the best model weights
)

# Learning Rate Scheduler Callback
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",  # Monitor validation loss
    factor=0.1,          # Reduce learning rate by a factor of 0.1
    patience=2           # Wait 2 epochs before reducing the learning rate
)

# Model Checkpoint Callback to save the best model
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="angry_expression_model_best.h5",  # Filepath to save the best model
    monitor="val_loss",  # Monitor validation loss
    save_best_only=True,  # Save only when the monitored metric improves
    save_weights_only=False,  # Save the full model
    mode="min",  # Minimize validation loss
    verbose=1  # Print log when saving
)

In [29]:
# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    verbose=1,  # Set to 1 for detailed logs
    callbacks=[early_stopping, lr_scheduler, model_checkpoint]  # Include callbacks
)

print("[INFO] Model training completed. Best model saved as 'angry_expression_model_best.h5'.")

Epoch 1/20
23581/23581 [==============================] - ETA: 0s - loss: 0.0078 - accuracy: 0.9976
Epoch 1: val_loss improved from inf to 0.00190, saving model to angry_expression_model_best.h5
23581/23581 [==============================] - 12699s 538ms/step - loss: 0.0078 - accuracy: 0.9976 - val_loss: 0.0019 - val_accuracy: 0.9995 - lr: 0.0010
Epoch 2/20
23581/23581 [==============================] - ETA: 0s - loss: 0.0031 - accuracy: 0.9992
Epoch 2: val_loss improved from 0.00190 to 0.00179, saving model to angry_expression_model_best.h5
23581/23581 [==============================] - 13982s 593ms/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 0.0018 - val_accuracy: 0.9994 - lr: 0.0010
Epoch 3/20
23581/23581 [==============================] - ETA: 0s - loss: 0.0026 - accuracy: 0.9994
Epoch 3: val_loss did not improve from 0.00179
23581/23581 [==============================] - 12444s 528ms/step - loss: 0.0026 - accuracy: 0.9994 - val_loss: 0.0025 - val_accuracy: 0.9996 - lr: 0.00

In [ ]:

# Plot training and validation accuracy
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training vs. Validation Accuracy")
plt.show()

# Plot training and validation loss
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.title("Training vs. Validation Loss")
plt.show()

NameError: name 'plt' is not defined